In [273]:
import csv
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score

Cleaning the dataset using NLTK modules

In [285]:
train = csv.reader(open("V1.4_Training.csv","rt", errors="ignore",encoding="utf-8"), delimiter=',')
train_X=[] #train_X is an array of "array of words".There will be an "array of words" corresponding to each document.
train_Y=[]
for row in train:
    train_X.append(word_tokenize(row[1]))
    train_Y.append(int(row[2]))
print(len(train_X))
print(len(train_Y))
print(set(train_Y))
train_X[0]

8500
8500
{0, 1}


['``',
 'Please',
 'enable',
 'removing',
 'language',
 'code',
 'from',
 'the',
 'Dev',
 'Center',
 '``',
 'language',
 'history',
 "''",
 'For',
 'example',
 'if',
 'you',
 'ever',
 'selected',
 '``',
 'ru',
 "''",
 'and',
 '``',
 'ru-ru',
 "''",
 'laguages',
 'and',
 'you',
 'published',
 'this',
 'xap',
 'to',
 'the',
 'Store',
 'then',
 'it',
 'causes',
 'Tile',
 'localization',
 'to',
 'show',
 'the',
 'en-us',
 '(',
 'default',
 ')',
 'tile',
 'localization',
 'which',
 'is',
 'bad',
 '.',
 "''"]

In [295]:
validation_X_Y=csv.reader(open("C:/Users/priya/OneDrive/MIDAS/Subtask-A/SubtaskA_Trial_Test_Labeled.csv","rt", errors="ignore",encoding="utf-8"), delimiter=',')
validation_X=[] #validation_X is an array of "array of words".There will be an "array of words" corresponding to each document.
validation_Y=[]
first_run=True
for row in validation_X_Y:
    #The first row of validation_X_Y contains no information so I'm skipping it.
    if first_run:
        first_run=False
        continue
    validation_X.append(word_tokenize(row[1]))
    validation_Y.append(int(row[2]))
print(len(validation_X))
print(len(validation_Y))
print(set(validation_Y))
validation_X[0]

592
592
{0, 1}


['I',
 "'m",
 'not',
 'asking',
 'Microsoft',
 'to',
 'Gives',
 'permission',
 'like',
 'Android',
 'so',
 'any',
 'app',
 'can',
 'take',
 'my',
 'data',
 ',',
 'but',
 'do',
 "n't",
 'keep',
 'it',
 'restricted',
 'like',
 'iPhone',
 '.']

In [340]:
test=csv.reader(open("SubtaskA_EvaluationData.csv","rt", errors="ignore",encoding="utf-8"), delimiter=',')
print(type(test))
test_X=[] #test_X is an array of "array of words".There will be an "array of words" corresponding to each document.
sent_list=[] #I will use this array to create my submission csv file
for row in test:
    test_X.append(word_tokenize(row[1]))
    sent_list.append([row[0],row[1]])
print(len(test_X))
print(len(sent_list))
test_X[0]


<class '_csv.reader'>
833
833


['This', 'would', 'enable', 'live', 'traffic', 'aware', 'apps', '.']

Creating a list of stop words and punctuations

In [248]:
from nltk.corpus import stopwords
import string
stops=list(stopwords.words('english'))
punctuations=list(string.punctuation)
stops+=punctuations
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

The lemmatizer needs pos tags in a different format than what we get from using pos_tag().
eg.For Noun, the lemmatizer needs "n" instead of "NN".
The function get_simple_pos converts the pos tags into the format required by the lemmatizer.

In [249]:
from nltk.corpus import wordnet
from nltk import pos_tag
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

Removing stop words and then lemmatizing each word.

In [250]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def clean_data(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [251]:
train_X_clean=[clean_data(document) for document in train_X]

In [296]:
validation_X_clean=[clean_data(document) for document in validation_X]

In [253]:
test_X_clean=[clean_data(document) for document in test_X]

I plan to use count vectorizer to get the data into the format required by sklearn classifiers.
So first I'm converting the data into the format required by count vectorizer.

In [254]:
train_X_clean_joined=[" ".join(document) for document in train_X_clean]
train_X_clean_joined[0:5]

["`` please enable remove language code dev center `` language history '' example ever select `` ru '' `` ru-ru '' laguages publish xap store cause tile localization show en-us default tile localization bad ''",
 "`` note .csproj file supportedcultures entry like supportedcultures de-de ru ru-ru /supportedcultures remove `` ru '' language code publish new xap version old xap version still remains store `` replaced unpublished '' ''",
 "`` wich mean new version fully replace old version cause serious problem 1 ''",
 "`` user still receive old xap version app ''",
 "`` store randomly give old xap new xap version app ''"]

In [297]:
validation_X_clean_joined=[" ".join(document) for document in validation_X_clean]
validation_X_clean_joined[0]

"'m ask microsoft gives permission like android app take data n't keep restrict like iphone"

In [256]:
test_X_clean_joined=[" ".join(document) for document in test_X_clean]
test_X_clean_joined[0]

'would enable live traffic aware apps'

In [257]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2),max_df=0.7)

In [258]:
train_X_final=count_vec.fit_transform(train_X_clean_joined)
count_vec.get_feature_names()
type(train_X_final)

scipy.sparse.csr.csr_matrix

In [298]:
validation_X_final=count_vec.transform(validation_X_clean_joined)
validation_X_final

<592x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 5416 stored elements in Compressed Sparse Row format>

In [260]:
test_X_final=count_vec.transform(test_X_clean_joined)
test_X_final

<833x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 6706 stored elements in Compressed Sparse Row format>

In the training dataset,out of 8500 samples: 2085 correspond to suggestion while 6415 correspond to non-suggestion. 
Clearly there is a class-imbalance.

In [299]:
# Penalize Algorithms (Cost-Sensitive Training)
# The idea is to use penalized learning algorithms that increase the cost of classification mistakes on the minority class.
clf = SVC(kernel='linear', class_weight='balanced',probability=True)
clf.fit(train_X_final,train_Y)
# Predict class probabilities
clf_proba= clf.predict_proba(validation_X_final)
clf_proba
# Keep only the positive class
clf_proba_pos = [p[1] for p in clf_proba]
print( roc_auc_score(validation_Y,clf_proba_pos) )

0.6951013513513513


In [305]:
# Using tree-based algorithms. Decision trees often perform well on imbalanced datasets because their hierarchical 
# structure allows them to learn signals from both classes.
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(train_X_final,train_Y)
rfc_proba= rfc.predict_proba(validation_X_final)
rfc_proba_pos = [p[1] for p in rfc_proba]
print( roc_auc_score(validation_Y,rfc_proba_pos) )

0.730357468955442


In [308]:
from sklearn.svm import SVC
svc=SVC(kernel='linear',probability=True)
svc.fit(train_X_final,train_Y)
svc_proba= svc.predict_proba(validation_X_final)
svc_proba_pos = [p[1] for p in svc_proba]
print( roc_auc_score(validation_Y,svc_proba_pos) )

0.6989648009495983


In [309]:
from sklearn.naive_bayes import MultinomialNB
MNB=MultinomialNB()
MNB.fit(train_X_final,train_Y)
MNB_proba= MNB.predict_proba(validation_X_final)
MNB_proba_pos = [p[1] for p in MNB_proba]
print( roc_auc_score(validation_Y,MNB_proba_pos) )

0.7154401022644266


In [310]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(train_X_final,train_Y)
log_proba= log.predict_proba(validation_X_final)
log_proba_pos = [p[1] for p in log_proba]
print( roc_auc_score(validation_Y,log_proba_pos) )

0.7269790905770636


In [311]:
from sklearn.model_selection import GridSearchCV

In [312]:
svc1 =SVC()
grid={'C':[1e2,1e3,5e3,1e4,5e4,1e5],'gamma':[1e-3,1e-4,5e-4,5e-3]}
abc=GridSearchCV(svc1,grid)
abc.fit(train_X_final,train_Y)
abc.best_estimator_

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [314]:
svc=SVC(C=100,gamma=0.001,probability=True)
svc.fit(train_X_final,train_Y)
svc_proba= svc.predict_proba(validation_X_final)
svc_proba_pos = [p[1] for p in svc_proba]
print( roc_auc_score(validation_Y,svc_proba_pos) )

0.732885546018992


In [293]:
df = pd.DataFrame(train_X_final.toarray())
df["output"]=pd.DataFrame(train_Y)
print(df.output.value_counts())
df.head()

0    6415
1    2085
Name: output, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,output
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Up-sample Minority Class

In [315]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df[df.output==0]
df_minority = df[df.output==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority,replace=True,n_samples=6415,random_state=1) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
print(df_upsampled.output.value_counts())
# Separate input features (X) and target variable (y)
Y = df_upsampled.output
X = df_upsampled.drop('output', axis=1)

1    6415
0    6415
Name: output, dtype: int64


Using upsampled training data instead of usual training data

In [317]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X,Y)
rfc_proba= rfc.predict_proba(validation_X_final)
rfc_proba_pos = [p[1] for p in rfc_proba]
print( roc_auc_score(validation_Y,rfc_proba_pos) )

0.7223395270270271


In [319]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(X,Y)
log_proba= log.predict_proba(validation_X_final)
log_proba_pos = [p[1] for p in log_proba]
print( roc_auc_score(validation_Y,log_proba_pos) )

0.7181108473338202


In [321]:
svc=SVC(C=100,gamma=0.001,probability=True)
svc.fit(X,Y)
svc_proba= svc.predict_proba(validation_X_final.todense())
svc_proba_pos = [p[1] for p in svc_proba]
print( roc_auc_score(validation_Y,svc_proba_pos) )

0.7274356281957634


Down sample majority class

In [322]:
# Down-sample Majority Class

# Separate majority and minority classes
df_majority = df[df.output==0]
df_minority = df[df.output==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2085,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
print(df_downsampled.output.value_counts())
Y1 = df_downsampled.output
X1 = df_downsampled.drop('output', axis=1)

1    2085
0    2085
Name: output, dtype: int64


Using downsampled training data instead of usual training data

In [326]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X1,Y1)
rfc_proba= rfc.predict_proba(validation_X_final)
rfc_proba_pos = [p[1] for p in rfc_proba]
print( roc_auc_score(validation_Y,rfc_proba_pos) )

0.7523283418553688


In [324]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(X1,Y1)
log_proba= log.predict_proba(validation_X_final)
log_proba_pos = [p[1] for p in log_proba]
print( roc_auc_score(validation_Y,log_proba_pos) )

0.7379017531044558


In [325]:
svc=SVC(C=100,gamma=0.001,probability=True)
svc.fit(X1,Y1)
svc_proba= svc.predict_proba(validation_X_final.todense())
svc_proba_pos = [p[1] for p in svc_proba]
print( roc_auc_score(validation_Y,svc_proba_pos) )

0.7441791453615779


I will use the random forest classifier trained on the downsampled data to make predictions on the testing data.

In [328]:
predictions=rfc.predict(test_X_final)
predictions

array([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

Creating the csv file for submission

In [343]:
filewriter = csv.writer(open("RanaPriyam.csv", "w+"))
i=0
for data in sent_list:
    data.append(predictions[i])
    i+=1
for data in sent_list:
                filewriter.writerow([data[0],data[1],data[2]])